# 获取单点、区域降雨量

In [19]:
import ee
import geemap

from datetime import datetime, timedelta

# 初始化Google Earth Engine
ee.Initialize()


In [20]:
# 定义区域
area_of_interest = ''

In [52]:
# 定义开始和结束日期
# 获取当前日期
today = datetime.now()

# 如果今天不是月初，则设置结束日期为上个月的最后一天
if today.day != 1:
    end_date = today.replace(day=1) - timedelta(days=1)
else:
    end_date = today
print(end_date)
start_date = end_date - timedelta(days=3*365)  # 三年前

def calculate_monthly_average(year, month):
    start = datetime(year, month, 1)
    end = datetime(year, month+1, 1) if month < 12 else datetime(year+1, 1, 1)
    
    # 加载IMERG降雨数据集
    precipitations = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
        .filterBounds(area_of_interest) \
        .filterDate(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')) \
        .select('precipitationCal')  # 选择降雨量估计波段

    # 计算平均降雨量
    mean_precipitation = precipitations.mean()

    # 使用reduceRegion方法计算区域内的平均降雨量
    mean_value = mean_precipitation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=area_of_interest,
        scale=10000  # 设置合适的比例尺
    ).getInfo()
    print(mean_value)

    # 检查是否存在'precipitation'键
    if 'precipitationCal' in mean_value:
        return mean_value['precipitationCal']
    else:
        return None  # 或者可以选择一个合适的默认值，如0
    

# 循环遍历每个月，计算平均降水量
current_year = start_date.year
current_month = start_date.month
monthly_averages = {}

while current_year < end_date.year or (current_year == end_date.year and current_month <= end_date.month):
    average = calculate_monthly_average(current_year, current_month)
    monthly_averages[f"{current_year}-{current_month:02d}"] = average
    current_month += 1
    if current_month > 12:
        current_month = 1
        current_year += 1

# 打印结果
for month, average in monthly_averages.items():
    print(f"Average Precipitation in {month} (mm/hr): {average}")


2023-12-31 11:04:20.450263
{'precipitationCal': 0.003615891548305813}
{'precipitationCal': 0.0012620362765014535}
{'precipitationCal': 0.04388288663343185}
{'precipitationCal': 0.011896009512141174}
{'precipitationCal': 0.174323763428424}
{'precipitationCal': 0.29560741322610684}
{'precipitationCal': 0.3161761869695298}
{'precipitationCal': 0.18930016676624367}
{'precipitationCal': 0.23449251866830828}
{'precipitationCal': 0.16997074969488718}
{'precipitationCal': 0.06120203382785559}
{'precipitationCal': 0.017802478108824817}
{'precipitationCal': 0.00981191639236755}
{'precipitationCal': 0.04710143806742525}
{'precipitationCal': 0.02298920987868446}
{'precipitationCal': 0.20868001006369782}
{'precipitationCal': 0.07987018211370892}
{'precipitationCal': 0.11018036908260265}
{'precipitationCal': 0.1768064021910081}
{'precipitationCal': 0.16035155111783936}
{'precipitationCal': 0.021676295146598173}
{'precipitationCal': 0.05481363999863567}
{'precipitationCal': 0.011535650598631987}
{'pr

In [56]:
import pandas as pd

# 创建一个空的DataFrame
df = pd.DataFrame(columns=['Year', 'Month', 'Average Precipitation'])

# 遍历monthly_averages字典，将数据添加到DataFrame中
for key, value in monthly_averages.items():
    year, month = key.split('-')
    new_row = pd.DataFrame({'Year': [int(year)], 'Month': [int(month)], 'Average Precipitation': [value]})
    df = pd.concat([df, new_row], ignore_index=True)

df['Average Precipitation'] = df['Average Precipitation']*30*24
df.to_csv(r'C:\Users\Runker\Desktop\Average_Precipitation.csv')
# # 按月份分组，并计算每个月的平均降雨量
# monthly_avg = df.groupby('Month')['Average Precipitation'].mean().reset_index()

# # 重命名列，以反映这是三年平均值
# monthly_avg.rename(columns={'Average Precipitation': 'Three-Year Average Precipitation'}, inplace=True)

# # 显示结果
# monthly_avg['Three-Year Average Precipitation']*30*24


In [36]:
# 定义开始和结束日期

# 获取当前日期
today = datetime.now()

# 如果今天不是月初，则设置结束日期为上个月的最后一天
if today.day != 1:
    cache_date = today.replace(day=1) - timedelta(days=1)
else:
    cache_date = today
print(cache_date)
end_date = cache_date - timedelta(days=3*365)
start_date = end_date - timedelta(days=6*365)  # 三年前
# 函数：计算指定月份的平均降水量
def calculate_monthly_average(year, month):
    start = datetime(year, month, 1)
    end = datetime(year, month+1, 1) if month < 12 else datetime(year+1, 1, 1)
    
    # 加载IMERG降雨数据集
    precipitations = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V06') \
        .filterBounds(area_of_interest) \
        .filterDate(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')) \
        .select('precipitation')  # 选择降雨量估计波段

    # 计算平均降雨量
    mean_precipitation = precipitations.mean()

    # 使用reduceRegion方法计算区域内的平均降雨量
    mean_value = mean_precipitation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=area_of_interest,
        scale=1000  # 设置合适的比例尺
    ).getInfo()

    print(mean_value)

    return mean_value['precipitation']

# 循环遍历每个月，计算平均降水量
current_year = start_date.year
current_month = start_date.month
monthly_averages = {}

while current_year < end_date.year or (current_year == end_date.year and current_month <= end_date.month):
    average = calculate_monthly_average(current_year, current_month)
    monthly_averages[f"{current_year}-{current_month:02d}"] = average
    current_month += 1
    if current_month > 12:
        current_month = 1
        current_year += 1

# 打印结果
for month, average in monthly_averages.items():
    print(f"Average Precipitation in {month} (mm/hr): {average}*30*24")


2023-12-31 10:49:02.709130
{'precipitation': 0.03901465555910539}
{'precipitation': 0.048604601565411104}
{'precipitation': 0.04094401267337552}
{'precipitation': 0.10318559696822047}
{'precipitation': 0.28234453809757415}
{'precipitation': 0.2594561338985764}
{'precipitation': 0.15904002937616238}
{'precipitation': 0.2618603591914799}
{'precipitation': 0.11821919890053162}
{'precipitation': 0.13775148894679756}
{'precipitation': 0.04295911431136832}
{'precipitation': 0.06579713809889681}
{'precipitation': 0.045630771945779906}
{'precipitation': 0.019985492582267367}
{'precipitation': 0.1353370023713345}
{'precipitation': 0.18288933644875482}
{'precipitation': 0.1766456636712619}
{'precipitation': 0.31920550192958014}
{'precipitation': 0.15470470076523574}
{'precipitation': 0.2224522599350561}
{'precipitation': 0.0608004890552369}
{'precipitation': 0.1268817157854295}
{'precipitation': 0.08457570894852483}
{'precipitation': 0.02958571777472751}
{'precipitation': 0.023691634611697147}
{

In [44]:
import pandas as pd

# 创建一个空的DataFrame
df = pd.DataFrame(columns=['Year', 'Month', 'Average Precipitation'])

# 遍历monthly_averages字典，将数据添加到DataFrame中
for key, value in monthly_averages.items():
    year, month = key.split('-')
    new_row = pd.DataFrame({'Year': [int(year)], 'Month': [int(month)], 'Average Precipitation': [value]})
    df = pd.concat([df, new_row], ignore_index=True)
# 按月份分组，并计算每个月的平均降雨量
monthly_avg = df.groupby('Month')['Average Precipitation'].mean().reset_index()

# 重命名列，以反映这是三年平均值
monthly_avg.rename(columns={'Average Precipitation': 'Three-Year Average Precipitation'}, inplace=True)

# 显示结果
monthly_avg['Three-Year Average Precipitation']*30*24


,Year,Month,Average Precipitation
0,2015,1,0.039015
1,2015,2,0.048605
2,2015,3,0.040944
3,2015,4,0.103186
4,2015,5,0.282345
...,...,...,...
67,2020,8,0.108552
68,2020,9,0.442236
69,2020,10,0.133462
70,2020,11,0.025427


In [47]:
# 按月份分组，并计算每个月的平均降雨量
monthly_avg = df.groupby('Month')['Average Precipitation'].mean().reset_index()

# 重命名列，以反映这是三年平均值
monthly_avg.rename(columns={'Average Precipitation': 'Three-Year Average Precipitation'}, inplace=True)

# 显示结果
monthly_avg['Three-Year Average Precipitation']*30*24

0      42.886606
1      31.152480
2      57.549631
3      91.417253
4     141.111436
5     229.019783
6     151.754569
7     124.449586
8     141.612518
9      96.971970
10     31.504976
11     23.254766
Name: Three-Year Average Precipitation, dtype: float64

In [50]:
monthly_avg['Three-Year Average Precipitation'] = monthly_avg['Three-Year Average Precipitation']*30*24

,Month,Three-Year Average Precipitation
0,1,30878.356383
1,2,22429.785565
2,3,41435.734127
3,4,65820.422015
4,5,101600.234119
5,6,164894.243499
6,7,109263.289372
7,8,89603.702078
8,9,101961.012917
9,10,69819.818702
